In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
df2 = pd.read_csv('../movies-data/combine_data.csv')

C:\Users\vuhoa\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
refined_dataset = df2.groupby(by=['userId','title'], as_index=False).agg({"rating":"mean"})

refined_dataset.head()
refined_dataset.shape

(10901225, 3)

In [4]:
refined_dataset.info

<bound method DataFrame.info of           userId                                              title  rating
0              1               Confession of a Child of the Century     4.0
1              1                                      Fools Rush In     4.0
2              1                                     License to Wed     4.0
3              1                                       Rocky Balboa     5.0
4              1  Shriek If You Know What I Did Last Friday the ...     4.0
...          ...                                                ...     ...
10901220  270896                                             Waiter     4.0
10901221  270896                                   We Own the Night     3.0
10901222  270896                                  Who Killed Bambi?     3.0
10901223  270896                                         Will Penny     3.0
10901224  270896                                 Young and Innocent     5.0

[10901225 rows x 3 columns]>

In [5]:
# pivot and create movie-user matrix
df = refined_dataset.pivot(
     index='title',
   columns='userId',
      values='rating').fillna(0)

df.head()

MemoryError: Unable to allocate 11.1 GiB for an array with shape (265458, 5591) and data type float64

In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=3)

In [ ]:
indices

In [ ]:
distances

In [ ]:
# get the index for 'movie_0'
index_for_movie = df.index.tolist().index('Sleepless in Seattle')
# find the indices for the similar movies
sim_movies = indices[index_for_movie].tolist()
# distances between 'movie_0' and the similar movies
movie_distances = distances[index_for_movie].tolist()
# the position of 'movie_0' in the list sim_movies
id_movie = sim_movies.index(index_for_movie)
# remove 'movie_0' from the list sim_movies
sim_movies.remove(index_for_movie)
# remove 'movie_0' from the list movie_distances
movie_distances.pop(id_movie)
print('The Nearest Movies to movie_0:', sim_movies)
print('The Distance from movie_0:', movie_distances)

In [ ]:
# copy df
df1 = df.copy()

# find the nearest neighbors using NearestNeighbors(n_neighbors=3)
number_neighbors = 3
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

# convert user_name to user_index
user_index = df.columns.tolist().index(1)

# t: movie_title, m: the row number of t in df
for m,t in list(enumerate(df.index)):
  
  # find movies without ratings by user_4
  if df.iloc[m, user_index] == 0:
    sim_movies = indices[m].tolist()
    movie_distances = distances[m].tolist()
    
    if m in sim_movies:
      id_movie = sim_movies.index(m)
      sim_movies.remove(m)
      movie_distances.pop(id_movie) 

    
    else:
      sim_movies = sim_movies[:number_neighbors-1]
      movie_distances = movie_distances[:number_neighbors-1]
        
    # movie_similarty = 1 - movie_distance    
    movie_similarity = [1-x for x in movie_distances]
    movie_similarity_copy = movie_similarity.copy()
    nominator = 0

    # for each similar movie
    for s in range(0, len(movie_similarity)):
      
      # check if the rating of a similar movie is zero
      if df.iloc[sim_movies[s], user_index] == 0:

        # if the rating is zero, ignore the rating and the similarity in calculating the predicted rating
        if len(movie_similarity_copy) == (number_neighbors - 1):
          movie_similarity_copy.pop(s)
          
        else:
          movie_similarity_copy.pop(s-(len(movie_similarity)-len(movie_similarity_copy)))

      # if the rating is not zero, use the rating and similarity in the calculation
      else:
        nominator = nominator + movie_similarity[s]*df.iloc[sim_movies[s],user_index]

    # check if the number of the ratings with non-zero is positive
    if len(movie_similarity_copy) > 0:
      
      # check if the sum of the ratings of the similar movies is positive.
      if sum(movie_similarity_copy) > 0:
        predicted_r = nominator/sum(movie_similarity_copy)

      else:
        predicted_r = 0

    # if all the ratings of the similar movies are zero, then predicted rating should be zero
    else:
      predicted_r = 0

  # place the predicted rating into the copy of the original dataset
    df1.iloc[m,user_index] = predicted_r

In [ ]:
def recommend_movies(user, num_recommended_movies):

  print('The list of the Movies {} Has Watched \n'.format(user))

  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_movies = []

  for m in df[df[user] == 0].index.tolist():

    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_movies.append((m, predicted_rating))

  sorted_rm = sorted(recommended_movies, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Movies \n')
  rank = 1
  for recommended_movie in sorted_rm[:num_recommended_movies]:
    print('{}: {} - predicted rating:{}'.format(rank, recommended_movie[0], recommended_movie[1]))
    rank = rank + 1


In [ ]:
from pprint import pprint
print("Enter user id")
user= int(input())
print("Enter number of movies to be recommended:")
num_recommended_movies = int(input())

recommend_movies(user,num_recommended_movies)